In [15]:
#!apt install wget
!pip install bert-tensorflow

You should consider upgrading via the 'pip install --upgrade pip' command.


In [16]:
import tensorflow as tf
#tf.enable_eager_execution()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import numpy as np

In [17]:
with open('data/counts_1grams.txt') as fopen:
    f = fopen.read().split('\n')[:-1]
    
words = {}
for l in f:
    w, c = l.split('\t')
    c = int(c)
    words[w] = c + words.get(w, 0)


In [18]:
# original from https://github.com/cbaziotis/ekphrasis/blob/master/ekphrasis/classes/spellcorrect.py
# improved it

import re
from collections import Counter

class SpellCorrector:
    """
    The SpellCorrector extends the functionality of the Peter Norvig's
    spell-corrector in http://norvig.com/spell-correct.html
    """
    REGEX_TOKEN = re.compile(r'\b[a-z]{2,}\b')

    def __init__(self, words):
        """
        :param corpus: the statistics from which corpus to use for the spell correction.
        """
        super().__init__()
        self.WORDS = words
        self.N = sum(self.WORDS.values())
              
    @staticmethod
    def tokens(text):
        return REGEX_TOKEN.findall(text.lower())

    def P(self, word):
        """
        Probability of `word`.
        """
        return self.WORDS[word] / self.N

    def most_probable(self, words):
        _known = self.known(words)
        if _known:
            return max(_known, key=self.P)
        else:
            return []

    @staticmethod
    def edit_step(word):
        """
        All edits that are one edit away from `word`.
        """
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word):
        """
        All edits that are two edits away from `word`.
        """
        return (e2 for e1 in self.edit_step(word)
                for e2 in self.edit_step(e1))

    def known(self, words):
        """
        The subset of `words` that appear in the dictionary of WORDS.
        """
        return set(w for w in words if w in self.WORDS)

    def edit_candidates(self, word, assume_wrong=False, fast=True):
        """
        Generate possible spelling corrections for word.
        """

        if fast:
            ttt = self.known(self.edit_step(word)) or {word}
        else:
            ttt = self.known(self.edit_step(word)) or self.known(self.edits2(word)) or {word}
        
        ttt = self.known([word]) | ttt
        return list(ttt)

In [19]:
corrector = SpellCorrector(words)

In [20]:
#possible_states = corrector.edit_candidates('eting')
possible_states = corrector.edit_candidates('gife')
possible_states

['rife', 'give', 'life', 'gibe', 'wife', 'gift']

In [21]:
BERT_VOCAB = 'uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = 'uncased_L-12_H-768_A-12/bert_config.json'

In [22]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

In [23]:
#text = 'scientist suggests eting burger can lead to obesity'
text = 'gife me something to eat'
#text_mask = text.replace('eting', '**mask**')
text_mask = text.replace('gife', '**mask**')
text_mask

'**mask** me something to eat'

In [24]:
def tokens_to_masked_ids(tokens, mask_ind):
    masked_tokens = tokens[:]
    masked_tokens[mask_ind] = "[MASK]"
    masked_tokens = ["[CLS]"] + masked_tokens + ["[SEP]"]
    masked_ids = tokenizer.convert_tokens_to_ids(masked_tokens)
    return masked_ids

In [25]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

In [26]:
bert_config

In [68]:
class Model:
    def __init__(self,):
        self.X = tf.placeholder(tf.int32, [None, None])
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=False,
            input_ids=self.X,
            use_one_hot_embeddings=False)
        
        self.output_layer = model.get_sequence_output()
        self.embedding = model.get_embedding_table()
        
#         print(modeling.get_activation(bert_config.hidden_act))
#         print(self.output_layer.shape)
#         print(self.embedding.shape)
#         print(bert_config.hidden_size)
#         print(bert_config.initializer_range)
        with tf.variable_scope('cls/predictions'):
            with tf.variable_scope('transform'):
                input_tensor = tf.layers.dense(
                    self.output_layer,
                    units = bert_config.hidden_size,
                    activation = modeling.get_activation(bert_config.hidden_act),
                    kernel_initializer = modeling.create_initializer(
                        bert_config.initializer_range
                    ),
                )
                input_tensor = modeling.layer_norm(input_tensor)
                print(input_tensor.shape)
            
            output_bias = tf.get_variable(
            'output_bias',
            shape = [bert_config.vocab_size],
            initializer = tf.zeros_initializer(),
            )
            logits = tf.matmul(input_tensor, self.embedding, transpose_b = True)
            self.logits = tf.nn.bias_add(logits, output_bias)

In [69]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


(?, ?, 768)


In [70]:
cls = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'cls')
cls

[<tf.Variable 'cls/predictions/transform/dense/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/dense/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/gamma:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/output_bias:0' shape=(30522,) dtype=float32_ref>]

In [71]:
saver = tf.train.Saver(var_list = var_lists + cls)

In [72]:
saver = tf.train.Saver(var_list = var_lists + cls)
#saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, BERT_INIT_CHKPNT)

In [38]:
replaced_masks = [text_mask.replace('**mask**', state) for state in possible_states]
replaced_masks

['rife me something to eat',
 'give me something to eat',
 'life me something to eat',
 'gibe me something to eat',
 'wife me something to eat',
 'gift me something to eat']

In [43]:
sent = replaced_masks[1]
sent

'give me something to eat'

In [45]:
tokens = tokenizer.tokenize(sent)
tokens

['give', 'me', 'something', 'to', 'eat']

In [47]:
input_ids = [tokens_to_masked_ids(tokens, i) for i in range(len(tokens))]
input_ids

[[101, 103, 2033, 2242, 2000, 4521, 102],
 [101, 2507, 103, 2242, 2000, 4521, 102],
 [101, 2507, 2033, 103, 2000, 4521, 102],
 [101, 2507, 2033, 2242, 103, 4521, 102],
 [101, 2507, 2033, 2242, 2000, 103, 102]]

In [75]:
preds = sess.run([model.embedding, 
                  model.output_layer,
                  tf.nn.softmax(model.logits)], feed_dict = {model.X: input_ids})

In [80]:
embedding = preds[0]
output_layer = preds[1]
embedding.shape



(30522, 768)

In [77]:
output_layer.shape

(5, 7, 768)